In [2]:
import numpy as np
import pandas as pd

from arch import arch_model
from arch.univariate import EGARCH
import matplotlib.pyplot as plt
import statsmodels.api as sm

# local module
from data_cleaning.import_data import import_data
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

In [29]:
# estimate and report parameter estimates for all pairs
pairs = {'USDRUB':'2005-03-03', 'USDZAR':'2003-02-23', 'USDBRL':'2003-10-01', 'EURUSD':'2000-03-15', 'USDJPY':'2000-02-23', 'GBPUSD':'2000-02-23'}
params = pd.DataFrame(index=['omega', 'alpha', 'beta', 'nu'])

for pair in pairs:
    df = import_data(fx_pair=pair)
    ret = df['log_ret']
    ret = ret.loc[pairs[pair]:'2020-12-31'] * 100
    #ret[0] = 0

    # fit model
    garch = arch_model(ret, mean='Zero', vol='GARCH', p=1,q=1, dist = 'Studentst')
    garch_fit = garch.fit(update_freq = 0, disp = 'off')

    # fetch params
    params_it = []
    params_it.append(garch_fit.params['omega'])
    params_it.append(garch_fit.params['alpha[1]'])
    params_it.append(garch_fit.params['beta[1]'])
    params_it.append(garch_fit.params['nu'])

    params[pair] = params_it

params

,USDRUB,USDZAR,USDBRL,EURUSD,USDJPY,GBPUSD
omega,0.000894,0.019531,0.009948,0.000738,0.003344,0.002617
alpha,0.069400,0.059770,0.103900,0.035926,0.057460,0.041727
beta,0.930601,0.923261,0.892023,0.962593,0.935844,0.950618
nu,5.731351,10.306982,7.680272,10.001919,5.444492,8.650045
